# Migrate Single Database


# Initialization
You will have to both of the 2 code cells in this section each time you reconnect to the cluster.

## Import Collector Library
Note: The usage of "from ... import" works expects a single .py file, as included from github.
If you are not using github repos, create a notebook with the DbInventoryCollector.py file's contents in it, and change this line to read:

```%run ./DB-Inventory-Collector```

In [ ]:
from DbInventoryCollector import InventoryCollector

In [ ]:
InventoryCollector.CreateWidgets(dbutils, spark, reset=False)

#Initialize InventoryCollector
collector = InventoryCollector(spark, dbutils.widgets.get("Inventory_Catalog"), dbutils.widgets.get("Inventory_Database"))
collector.initialize()

# Single Database Migration

## DDL Generation

In [ ]:
sourceCatalog = dbutils.widgets.get("Scan_Catalog")
sourceDatabase = dbutils.widgets.get("Scan_Database")
destCatalog = dbutils.widgets.get("Migration_Catalog")


if destCatalog == "ChangeMeToDest":
    raise Exception("You must set the destination catalog using the widgets above")

(ddl_objects, ddl_grants) = collector.generate_migration_ddl(sourceCatalog, sourceDatabase, destCatalog)

print("Finished Generation of both object and grant DDL")

## Inspect Generated DDL

In [ ]:
print(';\n\n'.join(ddl_objects))

In [ ]:
print(';\n\n'.join(ddl_grants))

## Execute DDL

In [ ]:
print("Executing Object Creation DDL")
collector.execute_sql_list(ddl_objects)
print("Executing Grants")
collector.execute_sql_list(ddl_grants)